# Koneksi Database

In [2]:
import sqlite3
import pandas as pd

# Membuat Objek untuk Execute SQL

In [3]:
db = sqlite3.connect('fuzzy.sqlite')
sql = db.cursor()

# Data Awal

In [4]:
command = "SELECT * FROM karyawan"
# for row in sql.fetchall() :
#     print(row[0], " ", row[1], " ", row[2], " ", row[3], " ", row[4])

sql.execute(command)
results = sql.fetchall()
cols = [column[0] for column in sql.description]
results_data= pd.DataFrame.from_records(results, columns = cols)
results_data    

,nip,nama,umur,masakerja,gaji
0,01,Lia,30,6.0,750000.0
1,02,Iwan,48,17.0,1500000.0
2,03,Sari,36,14.0,1255000.0
3,04,Andi,37,4.0,1040000.0
4,05,Budi,42,12.0,950000.0
5,06,Amir,39,13.0,1600000.0
6,07,Rian,37,5.0,1250000.0
7,08,Kiki,32,1.0,550000.0
8,09,Alda,35,3.0,735000.0
9,10,Yoga,25,2.0,860000.0


# Fungsi Fuzzy Umur         

In [5]:
def _muda(umur):
    if umur <= 30:
        return 1
    if umur >= 40:
        return 0
    if umur > 30 and umur < 40:
        return (40 - umur) / 10

def _parobaya(umur):
    if umur <= 35 or umur >= 50:
        return 0
    if umur > 35 and umur < 45:
        return (umur - 35)/ 10
    if umur > 45 and umur < 50:
        return (50 - umur) / 5
def _tua(umur):
    if umur <= 40:
        return 0
    if umur >= 50:
        return 1
    if umur > 40 and umur < 50:
        return (umur - 40) / 10

def predict_Umur(umur):
    muda = _muda(umur)
    parobaya = _parobaya(umur)
    tua = _tua(umur)

    return muda, parobaya, tua

# Fungsi Fuzzy Masa Kerja

In [6]:
def _baru(masakerja):
    if masakerja <= 5:
        return 1
    if masakerja >= 15:
        return 0
    if masakerja > 5 or masakerja < 15:
        return (15 - masakerja)/10


def _lama(masakerja):
    if masakerja <= 10:
        return 0
    if masakerja >= 25:
        return 1
    if masakerja > 10 or masakerja < 25:
        return (masakerja - 10)/15

def predict_MasaKerja(masakerja):
    baru = _baru(masakerja)
    lama = _lama(masakerja)
    return baru, lama

# Fungsi Fuzzy Gaji

In [7]:
def _rendah(gaji):
    if gaji <= 300000:
        return 1
    if gaji >=800000:
        return 0
    if gaji > 300000 or gaji < 800000:
        return (800000 - gaji) / 500000

def _sedang(gaji):
    if gaji <= 500000 or gaji >= 1500000:
        return 0
    if gaji> 500000 and gaji < 1000000:
        return (gaji - 500000)/ 500000
    if gaji > 1000000 and gaji < 1500000:
        return (1500000 - gaji) / 500000

def _tinggi(gaji):
    if gaji <= 1000000:
        return 0
    if gaji >=2000000:
        return 1
    if gaji > 1000000 or gaji < 2000000:
        return (gaji - 1000000) / 1000000

def predict_Gaji(gaji):       
    rendah = _rendah(gaji)
    sedang = _sedang(gaji)
    tinggi = _tinggi(gaji)
    return rendah, sedang, tinggi

# Fuzzy Umur

In [8]:
sql.execute("SELECT * FROM karyawan")

table_umur = pd.DataFrame({
  'Nama': [] ,
  'Umur' : [] ,
  'Muda': [] ,
  'Parobaya': [] ,
  ' Tua': [] 
})


for row in sql.fetchall() :
    muda, parobaya, tua = predict_Umur(row[2])

    table_umur.loc[len(table_umur.index)] = [row[1], row[2], muda, parobaya, tua]

table_umur




,Nama,Umur,Muda,Parobaya,Tua
0,Lia,30.0,1.0,0.0,0.0
1,Iwan,48.0,0.0,0.4,0.8
2,Sari,36.0,0.4,0.1,0.0
3,Andi,37.0,0.3,0.2,0.0
4,Budi,42.0,0.0,0.7,0.2
5,Amir,39.0,0.1,0.4,0.0
6,Rian,37.0,0.3,0.2,0.0
7,Kiki,32.0,0.8,0.0,0.0
8,Alda,35.0,0.5,0.0,0.0
9,Yoga,25.0,1.0,0.0,0.0


# Fuzzy Masa Kerja

In [9]:
sql.execute("SELECT * FROM karyawan")
table_masakerja = pd.DataFrame({
  'Nama': [] ,
  'Masa Kerja' : [] ,
  'Baru': [] ,
  'Lama': []
})


for row in sql.fetchall() :
    baru, lama = predict_MasaKerja(row[3])
    table_masakerja.loc[len(table_masakerja.index)]  = [row[1], row[3], baru, lama]

table_masakerja

,Nama,Masa Kerja,Baru,Lama
0,Lia,6.0,0.9,0.000000
1,Iwan,17.0,0.0,0.466667
2,Sari,14.0,0.1,0.266667
3,Andi,4.0,1.0,0.000000
4,Budi,12.0,0.3,0.133333
5,Amir,13.0,0.2,0.200000
6,Rian,5.0,1.0,0.000000
7,Kiki,1.0,1.0,0.000000
8,Alda,3.0,1.0,0.000000
9,Yoga,2.0,1.0,0.000000


# Fuzzy Gaji

In [10]:
sql.execute("SELECT * FROM karyawan")

header_gaji = ["Nama","Gaji","Rendah","Sedang","Tinggi"]
table_gaji = pd.DataFrame(columns=header_gaji)

for row in sql.fetchall() :
    rendah, sedang, tinggi = predict_Gaji(row[4])
    table_gaji.loc[len(table_gaji.index)] = [row[1], row[4], rendah, sedang, tinggi]

table_gaji

,Nama,Gaji,Rendah,Sedang,Tinggi
0,Lia,750000.0,0.10,0.50,0
1,Iwan,1500000.0,0.00,0.00,0.5
2,Sari,1255000.0,0.00,0.49,0.255
3,Andi,1040000.0,0.00,0.92,0.04
4,Budi,950000.0,0.00,0.90,0
5,Amir,1600000.0,0.00,0.00,0.6
6,Rian,1250000.0,0.00,0.50,0.25
7,Kiki,550000.0,0.50,0.10,0
8,Alda,735000.0,0.13,0.47,0
9,Yoga,860000.0,0.00,0.72,0


# Fuzzy umur muda dan gaji tinggi

In [11]:
sql.execute("SELECT * FROM karyawan")

header = ["Nama","Umur","Gaji","Muda","Tinggi","Muda & Tinggi"]
table = pd.DataFrame(columns=header)

for row in sql.fetchall() :
    muda, parobaya, tua = predict_Umur(row[2])
    rendah, sedang, tinggi = predict_Gaji(row[4])
    table.loc[len(table.index)] = [row[1],row[2],row[4],muda,tinggi,min(muda,tinggi)]

table

,Nama,Umur,Gaji,Muda,Tinggi,Muda & Tinggi
0,Lia,30,750000.0,1,0,0
1,Iwan,48,1500000.0,0,0.5,0
2,Sari,36,1255000.0,0.4,0.255,0.255
3,Andi,37,1040000.0,0.3,0.04,0.04
4,Budi,42,950000.0,0,0,0
5,Amir,39,1600000.0,0.1,0.6,0.1
6,Rian,37,1250000.0,0.3,0.25,0.25
7,Kiki,32,550000.0,0.8,0,0
8,Alda,35,735000.0,0.5,0,0
9,Yoga,25,860000.0,1,0,0


# Fuzzy umur muda atau gaji tinggi

In [12]:
sql.execute("SELECT * FROM karyawan")

header = ["Nama","Umur","Gaji","Muda","Tinggi","Muda / Tinggi"]
table = pd.DataFrame(columns=header)
for row in sql.fetchall() :
    muda, parobaya, tua = predict_Umur(row[2])
    rendah, sedang, tinggi = predict_Gaji(row[4])
    table.loc[len(table.index)] = [row[1],row[2],row[4],muda,tinggi,max(muda,tinggi)]

table

,Nama,Umur,Gaji,Muda,Tinggi,Muda / Tinggi
0,Lia,30,750000.0,1,0,1
1,Iwan,48,1500000.0,0,0.5,0.5
2,Sari,36,1255000.0,0.4,0.255,0.4
3,Andi,37,1040000.0,0.3,0.04,0.3
4,Budi,42,950000.0,0,0,0
5,Amir,39,1600000.0,0.1,0.6,0.6
6,Rian,37,1250000.0,0.3,0.25,0.3
7,Kiki,32,550000.0,0.8,0,0.8
8,Alda,35,735000.0,0.5,0,0.5
9,Yoga,25,860000.0,1,0,1
